# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [1]:
from selenium import webdriver  # permite criar o navegador
from selenium.webdriver.common.keys import Keys  # permite escrever no navegador
from selenium.webdriver.common.by import By  # permite selecionar items no navegador
import pandas as pd

In [2]:
# abrir o navegador
navegador = webdriver.Chrome()

# Passo 1: Pegar a cotação do dólar
# entrar no google
navegador.get('https://www.google.com/')

# pesquisar no google por "cotação dólar"
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dólar')
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar a contação que tá no google
cotacao_dolar = float(navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value'))
print(f'Dólar vale: R${cotacao_dolar:.2f}')

# Passo 2: Pegar a cotação do euro
# entrar no google
navegador.get('https://www.google.com/')

# pesquisar no google por "cotação euro"
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element('xpath', '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)

# pegar a contação que tá no google
cotacao_euro = float(navegador.find_element('xpath', '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value'))
print(f'Euro vale: R${cotacao_euro:.2f}')

# Passo 3: Pegar a cotação do ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')

# pegar a cotação que tá no site
cotacao_ouro = navegador.find_element('xpath', '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = float(cotacao_ouro.replace(',', '.'))
print(f'Ouro vale: R${cotacao_ouro:.2f}')

navegador.quit()

Dólar vale: R$5.20
Euro vale: R$5.15
Ouro vale: R$288.46


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [3]:
# Passo 4: Atualizar a base de dados (atualizando o preço de compra e o de venda)
df = pd.read_excel('Produtos.xlsx')
display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [4]:
# atualizar a coluna de cotação
df.loc[df['Moeda'] == 'Dólar', 'Cotação'] = cotacao_dolar
df.loc[df['Moeda'] == 'Euro', 'Cotação'] = cotacao_euro
df.loc[df['Moeda'] == 'Ouro', 'Cotação'] = cotacao_ouro

# atualizar a coluna de preço de compra
df.loc[df['Moeda'] == 'Dólar', 'Preço de Compra'] = df['Preço Original'] * df['Cotação']
df.loc[df['Moeda'] == 'Euro', 'Preço de Compra'] = df['Preço Original'] * df['Cotação']
df.loc[df['Moeda'] == 'Ouro', 'Preço de Compra'] = df['Preço Original'] * df['Cotação']

# atualizar a coluna de preço de venda
df['Preço de Venda'] = df['Preço de Compra'] * df['Margem']

In [5]:
display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.196700,5196.648033,1.40,7275.307246
1,Carro Renault,4500.00,Euro,5.151955,23183.798477,2.00,46367.596953
2,Notebook Dell,899.99,Dólar,5.196700,4676.978033,1.70,7950.862656
3,IPhone,799.00,Dólar,5.196700,4152.163300,1.70,7058.677610
4,Carro Fiat,3000.00,Euro,5.151955,15455.865651,1.90,29366.144737
5,Celular Xiaomi,480.48,Dólar,5.196700,2496.910416,2.00,4993.820832
6,Joia 20g,20.00,Ouro,288.460000,5769.200000,1.15,6634.580000


### Formatando valores no dataframe

In [6]:
df['Cotação'] = round(df['Cotação'], 2)
df['Preço de Compra'] = round(df['Preço de Compra'], 2)
df['Preço de Venda'] = round(df['Preço de Venda'], 2)

In [7]:
display(df)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.20,5196.65,1.40,7275.31
1,Carro Renault,4500.00,Euro,5.15,23183.80,2.00,46367.60
2,Notebook Dell,899.99,Dólar,5.20,4676.98,1.70,7950.86
3,IPhone,799.00,Dólar,5.20,4152.16,1.70,7058.68
4,Carro Fiat,3000.00,Euro,5.15,15455.87,1.90,29366.14
5,Celular Xiaomi,480.48,Dólar,5.20,2496.91,2.00,4993.82
6,Joia 20g,20.00,Ouro,288.46,5769.20,1.15,6634.58


### Agora vamos exportar a nova base de preços atualizada

In [8]:
# Passo 5: Exportar a base de preços atualizada
df.to_excel('Produtos Novo.xlsx', index=False)